<a href="https://colab.research.google.com/github/2303a52160/2303a52160-AIML_2025/blob/main/AIML_LAB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 Implement the AI Game Strategy

Part 1 (a). Install the Python Libraries required for Game Strategy

Install the python libraries- collections, random, math, functools, cache = functools.lru cache(10**6)

Implement a Game Class Constructor using action, is terminal, result, utility functions

A game is similar to a problem, but it has a terminal test instead of a goal test, and a utility for each terminal state.

Create a game subclass and implement actions, result, is terminal, and utility.

You will also need to set the initial attribute to the initial state; this can be done in the constructor.

In [ ]:
import collections
import random
import math
import functools

# Set the cache for memoization
cache = functools.lru_cache(maxsize=10**6)

class Game:
    def __init__(self, initial_state):
        self.initial = initial_state

    # Function to define possible actions for a given state
    def actions(self, state):
        """Returns the possible actions available in the given state."""
        raise NotImplementedError

    # Function to check if the game has reached a terminal state
    def is_terminal(self, state):
        """Returns True if the game has ended, False otherwise."""
        raise NotImplementedError

    # Function to return the state resulting from applying an action to the current state
    def result(self, state, action):
        """Returns the new state after applying the given action."""
        raise NotImplementedError

    # Function to return the utility value of the game (win/loss/draw) when a terminal state is reached
    def utility(self, state, player):
        """Returns the utility (value) of the terminal state for the player."""
        raise NotImplementedError

# Subclass of Game for a specific game (e.g., Tic-Tac-Toe)
class SimpleGame(Game):
    def __init__(self, initial_state):
        super().__init__(initial_state)

    # Define the possible actions in this game
    def actions(self, state):
        # Example: return available moves (assuming state is a list)
        return [i for i, val in enumerate(state) if val == 0]

    # Define the result of an action
    def result(self, state, action):
        new_state = state[:]
        new_state[action] = 1  # Example: Player 1's move
        return new_state

    # Check if the game is in a terminal state
    def is_terminal(self, state):
        return all(s != 0 for s in state)  # Example: all positions filled

    # Calculate the utility (e.g., 1 for win, -1 for loss, 0 for draw)
    def utility(self, state, player):
        # Simple utility function for this example
        if sum(state) > len(state) // 2:
            return 1 if player == 1 else -1
        return 0

# Example usage
initial_state = [0, 0, 0, 0, 0]  # Empty game state
game = SimpleGame(initial_state)

# Test the game functionality
print("Initial State:", game.initial)
print("Available Actions:", game.actions(game.initial))
next_state = game.result(game.initial, 2)
print("Next State:", next_state)
print("Is Terminal:", game.is_terminal(next_state))
print("Utility:", game.utility(next_state, 1))

Initial State: [0, 0, 0, 0, 0]
Available Actions: [0, 1, 2, 3, 4]
Next State: [0, 0, 1, 0, 0]
Is Terminal: False
Utility: 0


part-2
1. Implement the MiniMax Search Algorithm

In [ ]:
import math

def minimax_search(game, state):
    """Search the game tree to determine the best move; returns (value, move) pair."""
    player = state[1]  # 'X' or 'O', depending on whose turn it is

    # Max value function for maximizing player
    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    # Min value function for minimizing opponent
    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None

        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    # Return the best value and the corresponding move
    return max_value(state) if player == 'X' else min_value(state)

# Test the minimax with TicTacToe game
class TicTacToe(Game):
    def __init__(self):
        # Start with an empty 3x3 grid and 'X' going first
        super().__init__(initial_state=([' '] * 9, 'X'))

    # Available actions are the indices of empty cells
    def actions(self, state):
        board, player = state
        return [i for i, cell in enumerate(board) if cell == ' ']

    # Resulting state after placing 'X' or 'O' at position 'action'
    def result(self, state, action):
        board, player = state
        new_board = board.copy()
        new_board[action] = player
        next_player = 'O' if player == 'X' else 'X'
        return (new_board, next_player)

    # Terminal state if there's a winner or the board is full
    def is_terminal(self, state):
        board, _ = state
        return self.check_winner(board) or ' ' not in board

    # Utility: +1 for 'X' win, -1 for 'O' win, 0 for a draw
    def utility(self, state, player):
        board, _ = state
        winner = self.check_winner(board)
        if winner == 'X':
            return 1 if player == 'X' else -1
        elif winner == 'O':
            return 1 if player == 'O' else -1
        else:
            return 0

    # Helper function to check if there's a winner
    def check_winner(self, board):
        win_conditions = [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                          (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                          (0, 4, 8), (2, 4, 6)]             # Diagonals
        for (i, j, k) in win_conditions:
            if board[i] == board[j] == board[k] and board[i] != ' ':
                return board[i]
        return None

# Instantiate and run the minimax
tic_tac_toe_game = TicTacToe()
initial_state = tic_tac_toe_game.initial
best_value, best_move = minimax_search(tic_tac_toe_game, initial_state)
print(f"Best move: {best_move}, Value: {best_value}")


Best move: 0, Value: 0


.Implement Alpha-beta Pruning

In [ ]:
import math

# Define the game tree as a simple dictionary
# Each state is a key, and its possible actions and resulting states are defined
game_tree = {
    5: {'actions': ['increment', 'decrement']},
    6: {'actions': []},   # Terminal states
    4: {'actions': []},   # Terminal states
    10: {'actions': []},  # Terminal states with utility 10
    0: {'actions': []}    # Terminal states with utility -10
}

# Define utilities for terminal states
utilities = {
    10: 10,
    0: -10
}

def alpha_beta_search(initial_state):
    def max_value(state, alpha, beta):
        if state in utilities:
            return utilities[state]

        v = -math.inf
        for action in game_tree[state]['actions']:
            next_state = result(state, action)
            v = max(v, min_value(next_state, alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if state in utilities:
            return utilities[state]

        v = math.inf
        for action in game_tree[state]['actions']:
            next_state = result(state, action)
            v = min(v, max_value(next_state, alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    def result(state, action):
        if action == 'increment':
            return state + 1
        elif action == 'decrement':
            return state - 1
        else:
            raise ValueError("Invalid action")

    best_action = None
    alpha = -math.inf
    beta = math.inf

    for action in game_tree[initial_state]['actions']:
        next_state = result(initial_state, action)
        v = min_value(next_state, alpha, beta)
        if v > alpha:
            alpha = v
            best_action = action

    return best_action

# Example usage
initial_state = 5
best_action = alpha_beta_search(initial_state)
print(f"Best action from state {initial_state} is: {best_action}")


Best action from state 5 is: increment


Part 3 Implement the Game Strategy using TicTocToe

a. Implement TicToCToe game using display constructors init , actions, result, is terminal, utility,

In [ ]:
class TicTacToe:
    def __init__(self):
        """Initialize the game with an empty 3x3 board and 'X' as the first player."""
        self.board = [' '] * 9  # A list of 9 spaces representing the Tic-Tac-Toe grid
        self.current_player = 'X'  # 'X' always goes first

    def display(self):
        """Display the current state of the board."""
        print()
        for i in range(3):
            print(f" {self.board[3 * i]} | {self.board[3 * i + 1]} | {self.board[3 * i + 2]}")
            if i < 2:
                print("---|---|---")
        print()

    def actions(self):
        """Return the list of available actions (empty spaces) on the board."""
        return [i for i in range(len(self.board)) if self.board[i] == ' ']

    def result(self, action):
        """Return the new game state after performing the action."""
        new_game = TicTacToe()
        new_game.board = self.board[:]
        new_game.board[action] = self.current_player
        new_game.current_player = 'O' if self.current_player == 'X' else 'X'
        return new_game

    def is_terminal(self):
        """Check if the game has ended (win or draw)."""
        return self.check_winner() is not None or ' ' not in self.board

    def utility(self):
        """Return 1 if 'X' wins, -1 if 'O' wins, 0 for a draw."""
        winner = self.check_winner()
        if winner == 'X':
            return 1
        elif winner == 'O':
            return -1
        else:
            return 0

    def check_winner(self):
        """Check for a winner and return 'X', 'O', or None."""
        win_conditions = [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                          (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                          (0, 4, 8), (2, 4, 6)]             # Diagonals
        for (i, j, k) in win_conditions:
            if self.board[i] == self.board[j] == self.board[k] and self.board[i] != ' ':
                return self.board[i]
        return None

    def play(self):
        """Play a game of Tic-Tac-Toe."""
        while not self.is_terminal():
            self.display()
            print(f"Player {self.current_player}'s turn")
            action = self.get_action()
            self = self.result(action)

        self.display()
        winner = self.check_winner()
        if winner:
            print(f"Player {winner} wins!")
        else:
            print("It's a draw!")

    def get_action(self):
        """Get a valid action from the current player."""
        available_actions = self.actions()
        while True:
            try:
                action = int(input(f"Choose a move (0-8): "))
                if action in available_actions:
                    return action
                else:
                    print("Invalid move. Try again.")
            except ValueError:
                print("Please enter a number between 0 and 8.")

# Play the game
tic_tac_toe = TicTacToe()
tic_tac_toe.play()



   |   |  
---|---|---
   |   |  
---|---|---
   |   |  

Player X's turn

   |   | X
---|---|---
   |   |  
---|---|---
   |   |  

Player O's turn

   |   | X
---|---|---
   | O |  
---|---|---
   |   |  

Player X's turn

   | X | X
---|---|---
   | O |  
---|---|---
   |   |  

Player O's turn

   | X | X
---|---|---
   | O |  
---|---|---
 O |   |  

Player X's turn
Invalid move. Try again.
